In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [4]:
df_pyspark=spark.read.csv('test4.csv',header=True,inferSchema=True)

In [5]:
df_pyspark.show()

+--------+----+---+------+
|    Name|age |exp|salary|
+--------+----+---+------+
| Krishna|  31| 10| 30000|
|sudhashu|  30|  8| 25000|
|   sunny|  29|  4| 20000|
|    paul|  24|  3| 20000|
|  harsha|  21|  1| 15000|
| shubham|  23|  2| 18000|
+--------+----+---+------+



In [7]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age : integer (nullable = true)
 |-- exp: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
df_pyspark.columns

['Name', 'age ', 'exp', 'salary']

In [ ]:
## [Age,Experience]-->new featur-->indepent feature

In [14]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','exp'],outputCol="independent features")

In [16]:
from pyspark.ml.feature import VectorAssembler
# Use the correct column names including the leading space for 'age '
featureassembler=VectorAssembler(inputCols=['age ', 'exp'],outputCol="independent features")
output=featureassembler.transform(df_pyspark)

In [17]:
output.show()

+--------+----+---+------+--------------------+
|    Name|age |exp|salary|independent features|
+--------+----+---+------+--------------------+
| Krishna|  31| 10| 30000|         [31.0,10.0]|
|sudhashu|  30|  8| 25000|          [30.0,8.0]|
|   sunny|  29|  4| 20000|          [29.0,4.0]|
|    paul|  24|  3| 20000|          [24.0,3.0]|
|  harsha|  21|  1| 15000|          [21.0,1.0]|
| shubham|  23|  2| 18000|          [23.0,2.0]|
+--------+----+---+------+--------------------+



In [19]:
output.columns

['Name', 'age ', 'exp', 'salary', 'independent features']

In [20]:
finalised_data=output.select('independent features','salary')

In [21]:
finalised_data.show()

+--------------------+------+
|independent features|salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [22]:
from pyspark.ml.regression import LinearRegression
#train test splita
train_data,test_data=finalised_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='independent features',labelCol='salary')
regressor=regressor.fit(train_data)


In [23]:
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [24]:
regressor.intercept

16470.039946737463

In [25]:
pred_res=regressor.evaluate(test_data)

In [26]:
pred_res.predictions.show()

+--------------------+------+------------------+
|independent features|salary|        prediction|
+--------------------+------+------------------+
|          [30.0,8.0]| 25000|26903.595206391477|
+--------------------+------+------------------+



In [27]:
pred_res.meanAbsoluteError,pred_res.meanSquaredError

(1903.5952063914774, 3623674.7097966117)

In [31]:
pred_res.predictionCol

'prediction'